In [1]:
#Tratamento de exceção de import dos módulos
try: 
  import warnings    
  warnings.filterwarnings("ignore")
except ImportError:
   %pip install warnings
   import warnings   
   warnings.filterwarnings("ignore")

try:
  import pandas as pd
except ImportError:
    %pip install pandas
    import pandas

try:
  from tradingcomdados import alternative_data as ad
except ImportError:
    %pip install tradingcomdados
    from tradingcomdados import alternative_data as ad

try:
   import yahooquery as yf
   from yahooquery import Ticker
except ImportError:
   %pip install yahooquery --upgrade --no-cache-dir
   import yahooquery as yf
   from yahooquery import Ticker

try:
  import datetime as dt
except ImportError:
   %pip install datetime
   import datetime as dt

In [2]:
file = 'Inputs/MinhasAcoes.xlsx'

In [3]:
acoes_ibov = pd.read_excel(file, sheet_name='Planilha1') #minha tabela de ações

In [4]:
dfAcoes = pd.DataFrame(columns = ['SETORECONOMICO','SUBSETOR','SEGMENTO','EMPRESA','TITULO','ACAO','QTDECOTAS','VALOR','DATACOMPRA'])
dfAcoes = dfAcoes.astype( 
    dtype={
        'SETORECONOMICO': str,
        'SUBSETOR': str,
        'SEGMENTO':str,
        'EMPRESA':str,
        'TITULO':str,
        'ACAO':str,
        'QTDECOTAS':int,
        'VALOR':'Float64',
        'DATACOMPRA':str
        }
    )

In [5]:
for cod in acoes_ibov['CODIGO']:
    tkt = str(acoes_ibov['CODIGO'][acoes_ibov['CODIGO'] == cod].values[0])
    dfSect = ad.get_sectors(tkt)
    try:
        setor       = dfSect['SETOR ECONÔMICO'].values[0]
        subsetor    = dfSect['SUBSETOR'].values[0]
        segmento    = dfSect['SEGMENTO'].values[0]
        empresa     = dfSect['NOME NO PREGÃO'].values[0]
        titulo      = str(acoes_ibov['TITULO CORRETORA RICO'][acoes_ibov['CODIGO'] == cod].values[0])
        acao        = str(acoes_ibov['CODIGO'][acoes_ibov['CODIGO'] == cod].values[0])
        cotas       = str(acoes_ibov['COTAS'][acoes_ibov['CODIGO'] == cod].values[0])
        valor       = str(acoes_ibov['VALOR'][acoes_ibov['CODIGO'] == cod].values[0])
        dtcompra    = acoes_ibov['DATA'][acoes_ibov['CODIGO'] == cod].values[0]

        dfAcoes.loc[-1] = [setor, subsetor, segmento, empresa, titulo, acao, cotas, valor, dtcompra]
        dfAcoes.index = dfAcoes.index + 1
        dfAcoes = dfAcoes.sort_index()

    except:
        print('Erro na ação: ' + tkt)

Erro na ação: GETT3
Erro na ação: GSFI11
Erro na ação: MELI34


In [6]:
#Deduplicação de linhas e ordena pela coluna do nome da Ação.
dfAcoes = dfAcoes.drop_duplicates().sort_values(by=['ACAO']).reset_index()

In [7]:
historico_cotacao = pd.DataFrame(columns = ['ACAO','DATACOTACAO','VALORABERTURA','MAIORCOTACAO','MENORCOTACAO','VALORFECHAMENTO','VALORAJUSTADO','DATAPROCESSAMENTO'])

In [8]:
end_date = dt.datetime.today()

for i in dfAcoes.index:
    tkt = dfAcoes['ACAO'].values[i] + '.SA'
    dtcompra  = pd.to_datetime(dfAcoes['DATACOMPRA'].values[i],errors='ignore', format='%Y-%m-%d')
    end_date = dt.datetime.today()
    hist = Ticker(str(tkt)).history(start=dtcompra, end=end_date).reset_index()

    for x in hist.index:

        acao = tkt.replace('.SA','')
        dtcp = pd.to_datetime(hist['date'].values[x],errors='ignore', format='%Y-%m-%d')
        vlop = hist['open'].values[x]
        vlhg = hist['high'].values[x]
        vllw = hist['low'].values[x]
        vlcl = hist['close'].values[x]
        vlad = hist['adjclose'].values[x]

        historico_cotacao.loc[-1] = [acao, dtcp, vlop, vlhg, vllw, vlcl, vlad, end_date]
        historico_cotacao.index = historico_cotacao.index + 1
        historico_cotacao = historico_cotacao.sort_index()


In [9]:
# Consolidando Ações Detalhadas com as suas respectivas cotações
CotacaoConsolidadas = pd.merge(dfAcoes,historico_cotacao,on='ACAO')

In [10]:
CotacaoConsolidadas

,index,SETORECONOMICO,SUBSETOR,SEGMENTO,EMPRESA,TITULO,ACAO,QTDECOTAS,VALOR,DATACOMPRA,DATACOTACAO,VALORABERTURA,MAIORCOTACAO,MENORCOTACAO,VALORFECHAMENTO,VALORAJUSTADO,DATAPROCESSAMENTO
0,36,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,AMBEV S/A,AMBEVS/A ON,ABEV3,10,17.65,2019-06-06,2024-05-23,11.920000,12.070000,11.920000,11.960000,11.960000,2024-05-24 10:43:10.222001
1,36,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,AMBEV S/A,AMBEVS/A ON,ABEV3,10,17.65,2019-06-06,2024-05-22,11.980000,12.110000,11.930000,11.970000,11.970000,2024-05-24 10:43:10.222001
2,36,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,AMBEV S/A,AMBEVS/A ON,ABEV3,10,17.65,2019-06-06,2024-05-21,12.100000,12.250000,12.020000,12.020000,12.020000,2024-05-24 10:43:10.222001
3,36,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,AMBEV S/A,AMBEVS/A ON,ABEV3,10,17.65,2019-06-06,2024-05-20,12.330000,12.390000,12.050000,12.100000,12.100000,2024-05-24 10:43:10.222001
4,36,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,AMBEV S/A,AMBEVS/A ON,ABEV3,10,17.65,2019-06-06,2024-05-17,12.400000,12.410000,12.280000,12.380000,12.380000,2024-05-24 10:43:10.222001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32492,1,Materiais Básicos,Mineração,Minerais Metálicos,VALE,VALE ONNM,VALE3,29,54.72,2020-02-07,2020-02-13,52.220001,52.400002,51.560001,52.139999,34.945564,2024-05-24 10:45:22.283217
32493,1,Materiais Básicos,Mineração,Minerais Metálicos,VALE,VALE ONNM,VALE3,29,54.72,2020-02-07,2020-02-12,53.099998,53.380001,52.830002,53.070000,35.568871,2024-05-24 10:45:22.283217
32494,1,Materiais Básicos,Mineração,Minerais Metálicos,VALE,VALE ONNM,VALE3,29,54.72,2020-02-07,2020-02-11,51.150002,52.200001,51.020000,52.049999,34.885242,2024-05-24 10:45:22.283217
32495,1,Materiais Básicos,Mineração,Minerais Metálicos,VALE,VALE ONNM,VALE3,29,54.72,2020-02-07,2020-02-10,51.900002,51.900002,49.919998,50.189999,33.638626,2024-05-24 10:45:22.283217
